In [0]:
#@title   importing drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#@title   generator.py
#generator.py

import cv2
import numpy as np

from keras.preprocessing.image import img_to_array


def category_label(labels, dims, n_labels):
    x = np.zeros([dims[0], dims[1], n_labels])
    #print(labels)
    for i in range(dims[0]):
        for j in range(dims[1]):
          x[i, j, (int)(labels[i][j]/255)] = 1
    x = x.reshape(dims[0] * dims[1], n_labels)
    return x


def data_gen_small(img_dir, mask_dir, lists, batch_size, dims, n_labels):
    while True:
        ix = np.random.choice(np.arange(len(lists)), batch_size)
        imgs = []
        labels = []
        for i in ix:
            # images
            img_path = img_dir + lists.iloc[i, 0] + '.tif'
#            print( img_path )
            original_img = cv2.imread(img_path)[:, :, ::-1]
            resized_img = cv2.resize(original_img, (dims[0], dims[1]))
            array_img = img_to_array(resized_img)
#            print(array_img.size)
            imgs.append(array_img)
            # masks
            original_mask = cv2.imread(mask_dir + lists.iloc[i, 0] + '.tif')
            resized_mask = cv2.resize(original_mask, (dims[0], dims[1]))
#            print(resized_mask.shape)
            array_mask = category_label(resized_mask[:, :, 0], dims, n_labels)
            labels.append(array_mask)
        imgs = np.array(imgs)
        labels = np.array(labels)
        yield imgs, labels

Using TensorFlow backend.


In [0]:
#@title   layers.py
#layers.py

from tensorflow.keras import backend as K
from keras.layers import Layer


class MaxPoolingWithArgmax2D(Layer):

    def __init__(
            self,
            pool_size=(2, 2),
            strides=(2, 2),
            padding='same',
            **kwargs):
        super(MaxPoolingWithArgmax2D, self).__init__(**kwargs)
        self.padding = padding
        self.pool_size = pool_size
        self.strides = strides

    def call(self, inputs, **kwargs):
        padding = self.padding
        pool_size = self.pool_size
        strides = self.strides
        if K.backend() == 'tensorflow':
            ksize = [1, pool_size[0], pool_size[1], 1]
            padding = padding.upper()
            strides = [1, strides[0], strides[1], 1]
            output, argmax = tf.nn.max_pool_with_argmax(
                    inputs,
                    ksize=ksize,
                    strides=strides,
                    padding=padding)
        else:
            errmsg = '{} backend is not supported for layer {}'.format(
                    K.backend(), type(self).__name__)
            raise NotImplementedError(errmsg)
        argmax = K.cast(argmax, K.floatx())
        return [output, argmax]

    def compute_output_shape(self, input_shape):
        ratio = (1, 2, 2, 1)
        output_shape = [
                dim//ratio[idx]
                if dim is not None else None
                for idx, dim in enumerate(input_shape)]
        output_shape = tuple(output_shape)
        return [output_shape, output_shape]

    def compute_mask(self, inputs, mask=None):
        return 2 * [None]


class MaxUnpooling2D(Layer):
    def __init__(self, size=(2, 2), **kwargs):
        super(MaxUnpooling2D, self).__init__(**kwargs)
        self.size = size

    def call(self, inputs, output_shape=None):
        updates, mask = inputs[0], inputs[1]
        with tf.variable_scope(self.name):
            mask = K.cast(mask, 'int32')
            input_shape = tf.shape(updates, out_type='int32')
            #  calculation new shape
            if output_shape is None:
                output_shape = (
                        input_shape[0],
                        input_shape[1]*self.size[0],
                        input_shape[2]*self.size[1],
                        input_shape[3])
            self.output_shape1 = output_shape

            # calculation indices for batch, height, width and feature maps
            one_like_mask = K.ones_like(mask, dtype='int32')
            batch_shape = K.concatenate(
                    [[input_shape[0]], [1], [1], [1]],
                    axis=0)
            batch_range = K.reshape(
                    tf.range(output_shape[0], dtype='int32'),
                    shape=batch_shape)
            b = one_like_mask * batch_range
            y = mask // (output_shape[2] * output_shape[3])
            x = (mask // output_shape[3]) % output_shape[2]
            feature_range = tf.range(output_shape[3], dtype='int32')
            f = one_like_mask * feature_range

            # transpose indices & reshape update values to one dimension
            updates_size = tf.size(updates)
            indices = K.transpose(K.reshape(
                K.stack([b, y, x, f]),
                [4, updates_size]))
            values = K.reshape(updates, [updates_size])
            ret = tf.scatter_nd(indices, values, output_shape)
            return ret

    def compute_output_shape(self, input_shape):
        mask_shape = input_shape[1]
        return (
                mask_shape[0],
                mask_shape[1]*self.size[0],
                mask_shape[2]*self.size[1],
                mask_shape[3]
                )


In [0]:
#@title model.py

#model.py

from keras.models import Model
from keras.layers import Input
from keras.layers.core import Activation, Reshape
from keras.layers.convolutional import Convolution2D
from keras.layers.normalization import BatchNormalization

#from layers import MaxPoolingWithArgmax2D, MaxUnpooling2D


def segnet(
        input_shape,
        n_labels,
        kernel=3,
        pool_size=(2, 2),
        output_mode="softmax"):
    # encoder
    inputs = Input(shape=input_shape)

    conv_1 = Convolution2D(64, (kernel, kernel), padding="same")(inputs)
    conv_1 = BatchNormalization()(conv_1)
    conv_1 = Activation("relu")(conv_1)
    conv_2 = Convolution2D(64, (kernel, kernel), padding="same")(conv_1)
    conv_2 = BatchNormalization()(conv_2)
    conv_2 = Activation("relu")(conv_2)

    pool_1, mask_1 = MaxPoolingWithArgmax2D(pool_size)(conv_2)

    conv_3 = Convolution2D(128, (kernel, kernel), padding="same")(pool_1)
    conv_3 = BatchNormalization()(conv_3)
    conv_3 = Activation("relu")(conv_3)
    conv_4 = Convolution2D(128, (kernel, kernel), padding="same")(conv_3)
    conv_4 = BatchNormalization()(conv_4)
    conv_4 = Activation("relu")(conv_4)

    pool_2, mask_2 = MaxPoolingWithArgmax2D(pool_size)(conv_4)

    conv_5 = Convolution2D(256, (kernel, kernel), padding="same")(pool_2)
    conv_5 = BatchNormalization()(conv_5)
    conv_5 = Activation("relu")(conv_5)
    conv_6 = Convolution2D(256, (kernel, kernel), padding="same")(conv_5)
    conv_6 = BatchNormalization()(conv_6)
    conv_6 = Activation("relu")(conv_6)
    conv_7 = Convolution2D(256, (kernel, kernel), padding="same")(conv_6)
    conv_7 = BatchNormalization()(conv_7)
    conv_7 = Activation("relu")(conv_7)

    pool_3, mask_3 = MaxPoolingWithArgmax2D(pool_size)(conv_7)

    conv_8 = Convolution2D(512, (kernel, kernel), padding="same")(pool_3)
    conv_8 = BatchNormalization()(conv_8)
    conv_8 = Activation("relu")(conv_8)
    conv_9 = Convolution2D(512, (kernel, kernel), padding="same")(conv_8)
    conv_9 = BatchNormalization()(conv_9)
    conv_9 = Activation("relu")(conv_9)
    conv_10 = Convolution2D(512, (kernel, kernel), padding="same")(conv_9)
    conv_10 = BatchNormalization()(conv_10)
    conv_10 = Activation("relu")(conv_10)

    pool_4, mask_4 = MaxPoolingWithArgmax2D(pool_size)(conv_10)

    conv_11 = Convolution2D(512, (kernel, kernel), padding="same")(pool_4)
    conv_11 = BatchNormalization()(conv_11)
    conv_11 = Activation("relu")(conv_11)
    conv_12 = Convolution2D(512, (kernel, kernel), padding="same")(conv_11)
    conv_12 = BatchNormalization()(conv_12)
    conv_12 = Activation("relu")(conv_12)
    conv_13 = Convolution2D(512, (kernel, kernel), padding="same")(conv_12)
    conv_13 = BatchNormalization()(conv_13)
    conv_13 = Activation("relu")(conv_13)

    pool_5, mask_5 = MaxPoolingWithArgmax2D(pool_size)(conv_13)
    print("Build enceder done..")

    # decoder

    unpool_1 = MaxUnpooling2D(pool_size)([pool_5, mask_5])

    conv_14 = Convolution2D(512, (kernel, kernel), padding="same")(unpool_1)
    conv_14 = BatchNormalization()(conv_14)
    conv_14 = Activation("relu")(conv_14)
    conv_15 = Convolution2D(512, (kernel, kernel), padding="same")(conv_14)
    conv_15 = BatchNormalization()(conv_15)
    conv_15 = Activation("relu")(conv_15)
    conv_16 = Convolution2D(512, (kernel, kernel), padding="same")(conv_15)
    conv_16 = BatchNormalization()(conv_16)
    conv_16 = Activation("relu")(conv_16)

    unpool_2 = MaxUnpooling2D(pool_size)([conv_16, mask_4])

    conv_17 = Convolution2D(512, (kernel, kernel), padding="same")(unpool_2)
    conv_17 = BatchNormalization()(conv_17)
    conv_17 = Activation("relu")(conv_17)
    conv_18 = Convolution2D(512, (kernel, kernel), padding="same")(conv_17)
    conv_18 = BatchNormalization()(conv_18)
    conv_18 = Activation("relu")(conv_18)
    conv_19 = Convolution2D(256, (kernel, kernel), padding="same")(conv_18)
    conv_19 = BatchNormalization()(conv_19)
    conv_19 = Activation("relu")(conv_19)

    unpool_3 = MaxUnpooling2D(pool_size)([conv_19, mask_3])

    conv_20 = Convolution2D(256, (kernel, kernel), padding="same")(unpool_3)
    conv_20 = BatchNormalization()(conv_20)
    conv_20 = Activation("relu")(conv_20)
    conv_21 = Convolution2D(256, (kernel, kernel), padding="same")(conv_20)
    conv_21 = BatchNormalization()(conv_21)
    conv_21 = Activation("relu")(conv_21)
    conv_22 = Convolution2D(128, (kernel, kernel), padding="same")(conv_21)
    conv_22 = BatchNormalization()(conv_22)
    conv_22 = Activation("relu")(conv_22)

    unpool_4 = MaxUnpooling2D(pool_size)([conv_22, mask_2])

    conv_23 = Convolution2D(128, (kernel, kernel), padding="same")(unpool_4)
    conv_23 = BatchNormalization()(conv_23)
    conv_23 = Activation("relu")(conv_23)
    conv_24 = Convolution2D(64, (kernel, kernel), padding="same")(conv_23)
    conv_24 = BatchNormalization()(conv_24)
    conv_24 = Activation("relu")(conv_24)

    unpool_5 = MaxUnpooling2D(pool_size)([conv_24, mask_1])

    conv_25 = Convolution2D(64, (kernel, kernel), padding="same")(unpool_5)
    conv_25 = BatchNormalization()(conv_25)
    conv_25 = Activation("relu")(conv_25)

    conv_26 = Convolution2D(n_labels, (1, 1), padding="valid")(conv_25)
    conv_26 = BatchNormalization()(conv_26)
    conv_26 = Reshape(
            (input_shape[0]*input_shape[1], n_labels),
            input_shape=(input_shape[0], input_shape[1], n_labels))(conv_26)

    outputs = Activation(output_mode)(conv_26)
    print("Build decoder done..")

    model = Model(inputs=inputs, outputs=outputs, name="SegNet")

    return model
model = segnet((512, 512 ,3), 2,
            3, (2, 2), "softmax")
print(model.summary())


In [0]:
#@title training.py

#train.py


import pandas as pd
import tensorflow as tf

#from  model import segnet
#from generator import data_gen_small


def main():
    # set the necessary list
    train_list = pd.read_csv("/content/drive/My Drive/sarseg/Data/complete_dataset/crop/train_list.csv", header=None)
    val_list = pd.read_csv("/content/drive/My Drive/sarseg/Data/complete_dataset/crop/val_list.csv", header=None)

    # set the necessary directories
    trainimg_dir = "/content/drive/My Drive/sarseg/Data/complete_dataset/crop/images/"
    trainmsk_dir = "/content/drive/My Drive/sarseg/Data/complete_dataset/crop/gt/"
    valimg_dir = "/content/drive/My Drive/sarseg/Data/complete_dataset/crop/images/"
    valmsk_dir = "/content/drive/My Drive/sarseg/Data/complete_dataset/crop/gt/"

    train_gen = data_gen_small(trainimg_dir, trainmsk_dir,
            train_list, 8,
            [512, 512], 2)
    val_gen = data_gen_small(valimg_dir, valmsk_dir,
            val_list, 8,
            [512, 512], 2)

    model = segnet((512, 512 ,3), 2,
            3, (2, 2), "softmax")
    print(model.summary())

    model.compile(loss="categorical_crossentropy",
            optimizer="adadelta", metrics=["accuracy"])
    model.fit_generator(train_gen, steps_per_epoch=8,
            epochs=8, validation_data=val_gen,
            validation_steps=8)

    model.save_weights("/content/drive/My Drive/sarseg/"+str(00)+".hdf5")
    model.save("/content/drive/My Drive/sarseg/"+str(00)+".h5")
    print("save weight done..")


if __name__ == "__main__":
    main()


RuntimeError: ignored

In [0]:
#@title   loading model

import keras
import pandas as pd
import tensorflow as tf

from keras.models import load_model
model_test = keras.models.load_model('/content/drive/My Drive/sarseg/Data/modelout/run0.h5',
                                     custom_objects={'MaxPoolingWithArgmax2D': MaxPoolingWithArgmax2D, 'MaxUnpooling2D':MaxUnpooling2D})
#model_test = segnet((512, 512 ,3), 2,
#          3, (2, 2), "softmax")
model_test.load_weights('/content/drive/My Drive/sarseg/Data/modelout/run0.hdf5')
print('model loaded')


In [0]:
#@title   getting loaded model accuracy


val_list = pd.read_csv("/content/drive/My Drive/sarseg/Data/complete_dataset/crop/val_list.csv", header=None)

valimg_dir = "/content/drive/My Drive/sarseg/Data/complete_dataset/crop/images/"
valmsk_dir = "/content/drive/My Drive/sarseg/Data/complete_dataset/crop/gt/"

val_gen = data_gen_small(valimg_dir, valmsk_dir,
             val_list, 1,
            [512, 512], 2)

model_test.compile(loss="categorical_crossentropy",
            optimizer="adadelta", metrics=["accuracy"])
score = model_test.evaluate(val_gen, steps = 1)
print("%s: %.2f%%" % (model_test.metrics_names[1], score[1]*100))

In [0]:
#@title predicting output

valimg_dir = "/content/drive/My Drive/sarseg/Data/complete_dataset/crop/images/"
valmsk_dir = "/content/drive/My Drive/sarseg/Data/complete_dataset/crop/gt/"

test = np.zeros([1, 512, 512, 3])
test_image = cv2.imread(valimg_dir + 'austin100.tif')
tested = cv2.resize(test_image, (512, 512))
test[0] = img_to_array(tested)

mask = np.zeros([1, 512, 512, 3])
test_image = cv2.imread(valmsk_dir + 'austin100.tif')
tested = cv2.resize(test_image, (512, 512))
array_mask = category_label(tested[:, :, 0], (512, 512, 3), 2)

predicted_out = model_test.predict(test)

In [0]:
print(predicted_out)
print(array_mask)

In [0]:
#@title   making image arrays for matplot

array_mask.shape
mask = np.zeros([512, 512])
for i in range(512):
  for j in range(512):
    mask[i][j] = array_mask[i*512+j][0];

predicted = predicted_out[0]
predicted.shape
pred = np.zeros([512, 512])
for i in range(512):
  for j in range(512):
    pred[i][j] = 1-predicted[i*512+j][0];


In [0]:
#@title   plotting images

import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure()
ax1 = fig.add_subplot(2, 2, 1)
ax1.imshow(test[0].astype(int))
ax2 = fig.add_subplot(2, 2, 2)
ax2.imshow(mask, cmap= 'Greys')
ax3 = fig.add_subplot(2, 2, 3)
ax3.imshow(pred, cmap= 'Greys')

In [0]:
#@title   printing prediction accuracy

from keras import backend as K
total = 512*512
intersection = 0
for i in range(512):
  for j in range(512):
    if( mask[i][j] == pred[i][j] ):
        intersection = intersection + 1 
    
print(intersection/total)